# Exercise 3 - Julia:  Multi-class Classification and Neural Networks

<img src="http://dalab.github.io/dissolve-struct/images/multiclass.png">

## Part I: *Multi-class Classification with Logistic Regression*

Recognizing handwritten digits.  Though the source is not explicitly mentioned, it is just like the [MNIST database](https://en.wikipedia.org/wiki/MNIST_database).

Each value of $X$ will be a 20x20 grid of values representing the grayscale intensity at that location "unrolled" into a 400-dimensional vector.  Here is an example for the first number in our data, $X^{(1)}$:

$\hspace{1cm} X^{(1)} = \begin{bmatrix}x_{(1, 1)}^{(1)},\ x_{(1, 2)}^{(1)},\ \dots\ x_{(1, 20)}^{(1)} \\
                                    x_{(2, 1)}^{(1)},\ x_{(2, 2)}^{(1)},\ \dots\ x_{(2, 20)}^{(1)} \\
                                    \vdots \\ 
                                    x_{(20, 1)}^{(1)},\ x_{(20, 2)}^{(1)},\ \dots\ x_{(20, 20)}^{(1)} \\ 
                    \end{bmatrix}
\rightarrow \begin{bmatrix} x_1^{(1)},\ x_2^{(1)},\ \dots\ x_{400}^{(1)} \end{bmatrix}
\rightarrow (x^{(1)})^T$

Here is our collection of all of the numbers for $X$:

$\hspace{1cm} X = \begin{bmatrix} (x^{(1)})^T \\ (x^{(2)})^T \\ \vdots \\ (x^{(400)})^T \end{bmatrix}$

---

Beginning with package imports, data loading, and initial visualization

In [2]:
using DataFrames
using Plots
using MAT  # Loading in .mat files
using GLM  # For comparing answers
using Optim  # For optimizing thetas

INFO: Precompiling module MAT.
INFO: Recompiling stale cache file C:\Users\JeffM\.julia\lib\v0.5\GLM.ji for module GLM.
INFO: Recompiling stale cache file C:\Users\JeffM\.julia\lib\v0.5\Optim.ji for module Optim.


In [8]:
# Reading in our MATLAB matrix
file = matopen("ex3/ex3data1.mat")
X = read(file, "X") # note that this does NOT introduce a variable ``varname`` into scope
y = read(file, "y")
close(file)

### Vectorizing Logistic Regression

We'll be taking a matrix multiplication approach to vectorize both the cost function and the gradient for logistic regression.  My previous notebooks for assignment \#2 ([Python](http://nbviewer.jupyter.org/github/JeffMacaluso/Misc/blob/master/MachineLearningMOOC/Ex2%20Python%20-%20Logistic%20Regression.ipynb) | [R](http://nbviewer.jupyter.org/github/JeffMacaluso/Misc/blob/master/MachineLearningMOOC/Ex2%20R%20-%20Logistic%20Regression.ipynb) | [Julia](http://nbviewer.jupyter.org/github/JeffMacaluso/Misc/blob/master/MachineLearningMOOC/Ex2%20Julia%20-%20Logistic%20Regression.ipynb)) already use a vectorized approach, but I'll have a little more information describing what is happening below.

- **Note:** The assignment calls for creating the functions for unregularized logistic regression and later modifying them for regularization, but I'm just going to write the functions for regularized logistic regression right away since the regularization parameter is generally just an addition at the end of the equations.

To illustrate this vectorization process, here is how we vectorize our hypothesis, $h_\theta(x)$, using matrix multiplication with the axiom that $a^Tb = b^Ta$ if $a$ and $b$ are vectors:

Defining $X$ and $\theta$ as

$\hspace{1cm} X = \begin{bmatrix} (x^{(1)})^T \\ (x^{(2)})^T \\ \vdots \\ (x^{(m)})^T \end{bmatrix} \hspace{0.5cm}$ and $\hspace{0.5cm} \theta = \begin{bmatrix} \theta_0 \\ \theta_1 \\ \vdots \\ \theta_n \end{bmatrix}$

Computing matrix product $X\theta$:

$\hspace{1cm} X\theta = \begin{bmatrix} (x^{(1)})^T\theta \\ (x^{(2)})^T\theta \\ \vdots \\ (x^{(m)})^T\theta \end{bmatrix} \hspace{0.5cm} = \hspace{0.5cm} \begin{bmatrix} \theta^T(x^{(1)}) \\ \theta^T(x^{(2)}) \\ \vdots \\ \theta^T(x^{(m)}) \end{bmatrix}$

Basically, the main difference between the vectorized approach and non-vectorized approach is through linear algebra.  The non-vectorized approach would be element-wise (individually compute the cost function/gradient/etc. for each element individually), whereas the vectorized is through matrices.

#### Vectorized Cost Function

$J(\theta) = \frac{1}{m} \sum_{i=1}^m[-y^{(i)}log(h_\theta(x^{(i)})) - (1-y^{(i)})log(1-h_\theta(x^{(i)}))] + \frac{\lambda}{2m} \sum_{j=1}^n \theta_j^2$

#### Vectorized Gradient

$\frac{\partial J(\theta)}{\partial \theta_j} = \begin{cases} 
\hspace{0.25cm} \frac{1}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})x_j^{(i)} & \text{for}\ j = 0 \\
\Big(\frac{1}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})x_j^{(i)}\Big) + \frac{\lambda}{m}\theta_j & \text{for}\ j \geq 1
\end{cases}$

Finding optimal values for $\theta$

Using **[Julia's optimization fmin_cg]** function instead of the fmin function used in the previous exercise because it is more efficient at dealing with a large number of parameters.

#### One-vs-all Classification

In order to predict the digit for each $X^{(i)}$, we need to create a function to do one-vs-all classification.  This means training 10 models (one for each unique value of $y$) to estimate the class likelihood of each digit, and then assign each value of $X^{(i)}$ a digit.  

This function first finds the **optimal $\theta$ values** for each model and places them into a table:

|      | 0    | 1    | ...  | 9    |
|------|------|------|------|------|
|   0  |-8.014|-3.071| ...  |-5.507|
|   1  | 0.000| 0.000| ...  | 0.000|
| ...  | ...  | ...  | ...  | ...  |
| 401  | 0.000| 0.000| ...  | 0.000|


And then calculates the **class probabilities** for each value of $X^{(i)}$ and places them into a table:

|      | 0    | 1    | ...  | 9    |
|------|------|------|------|------|
|   0  | 0.999| 0.001| ...  | 0.001|
|   1  | 0.999| 0.001| ...  | 0.001|
| ...  | ...  | ...  | ...  | ...  |
| 5000 | 0.001| 0.001| ...  | 0.999|


From here, we just grab the column that has the highest value

Checking the accuracy

The exercise states that the training accuracy should be around 94.9%

**[Comment on accuracy]**.  The accuracy also fluctuates with both the number of iterations (we're keeping it at **[# iterations]** since it's still relatively quick) and the type of optimization algorithm (we're sticking with cg since it's what the exercise recommends).

Comparing against Julia's GLM logistic regression:

**[Comment on accuracy]**.  Our accuracy is **[lower/higher/the same]** in this case, but we can raise it with increasing our iterations or by trying other optimization algorithms.  Regardless, the accuracy here is largely meaningless since we're not splitting between training/testing/validation sets.

Let's look at what we misclassified:

**[Comment on mis-classifications]**

---

##  Part II: *Neural Networks*

Recognizing the same handwritten digits with neural networks

### Model Representation

Note: Image taken from [scikit-learn documentation](http://scikit-learn.org/stable/modules/neural_networks_supervised.html)

<img src="http://scikit-learn.org/stable/_images/multilayerperceptron_network.png" style="width: 300px">

**Input layer $(X_n)$:** 400 nodes (excluding bias node)

**Hidden layer $(a_k$ or $s_l)$ :** 25 nodes (in one layer ($L=1$), excluding bias node)

**Output layer $(f(X))$:** 10 nodes (binary for each digit)

In feed-forward propagation, each node for each step (after the input layer) is calculated multiplying the previous nodes by their respective weights (provided in this exercise, but will be calculated with backpropagation in the next exercise), summing these products, and then performing an activation function - the sigmoid function in this case.  Here is a mathematical representation:

$a = f(\sum_{i=0}^N w_i x_i)$

- Notation:
    - **$w_i$:** Weight
    - **$x_i$:** Node
    - **$f()$:** Activation function

Checking the accuracy

The exercise states that the training accuracy should be around 97.5%